# Drug Related Overdose Deaths
## Individual Project- Regression
### Natasha Rivers
#### July 14, 2021

<hr style="border:1px solid black"> </hr>

## Project Scope

**Methodology Used**: Regression

**Target Variable**: Age

**Primary Question**: Does age play a role in drug related deaths?

<hr style="border:1px solid black"> </hr>

## Executive Summary

- There is not a single indicator of what causes drug related deaths.
- OLS model performed best. Outperforming baseline, LassoLars and TweedieRegressor models

|Model                                | rmse_train   | rmse_validate  | r^2
|-------------------------------------|--------------|----------------|---------
|mean_baseline                        | 12.240       | 12.536         | 0.000
|OLS                                  | 12.119       | 12.381         | 0.024
|LassoLars (alpha 2)                  | 12.240       | 12.536         | 0.000
|TweedieRegressor (power 1, alpha 2)  | 12.121       | 12.384         | 0.024


<hr style="border:3px solid black"> </hr>

# Imports

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#custom modules
import acquire
import prepare

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, TweedieRegressor, LassoLars
from sklearn.metrics import mean_squared_error, explained_variance_score

<hr style="border:2px solid black"> </hr>

# Acquire

In [ ]:
#function from acquire.py
df = acquire.od_deaths_df()

In [ ]:
df.head()

In [ ]:
df.info()

### Takeaways from Acquire:
- imported the csv into Pandas and created a dataframe
- original dataframe contains:
    - 41 columns 
        - multiple data types
    - 5105 entries
- will likely need to:
    - drop some columns
    - rename columns
    - change data types
- dummies already appear to have been created
- contains many nulls

<hr style="border:2px solid black"> </hr>

# Prepare

In [ ]:
#look into nulls
df.isnull().sum()

In [ ]:
#look into variations of manner of death
df.MannerofDeath.value_counts()

In [ ]:
#look into Race combinations being represented
df.Race.value_counts()

In [ ]:
df.Location.value_counts()

In [ ]:
df = prepare.prep_set(df)

In [ ]:
df.head()

In [ ]:
#prep_set function from prepare.py
#filled all nulls and dropped unneccessary columns
df.isnull().sum()

In [ ]:
#prep_set function from prepare.py made results more uniform
df.MannerofDeath.value_counts()

In [ ]:
#prep_set function from prepare.py corrected Race choices into most common
df.Race.value_counts()

In [ ]:
#prep_set function from prepare.py corrected Location choices into most common
df.Location.value_counts()

In [ ]:
#split data using od_deaths_split function from prepare file
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = prepare.od_deaths_split(df, 'Age')

### Takeaways from Prepare:

- Created a function (prep_set) that did the following:  
    - dropped several columns with high null value 
        - were not significant if only had ~10% of data
    - filled null values by imputing
        - using mean and most frequent
    - changed data for uniformity in Manner of Death
    - corrected options for 'Race' so that it would not be so vast
    - corrected options for 'Location', 'Morphine_NotHeroin', 'AnyOpioid', and 'Fentanyl' to turn into Booleans
    - changed data type in 'Age' from float to integer
    - created dummy columns for race
    
<br>

- Created a function to split data 
    - split data into train, validate, test using Age as the target
    - prints out shape of all three sets
   
<br>

- Final DataFrame ended up with:
    - 5105 entries
    - 32 columns
        - all with int64 and object datatypes

<hr style="border:2px solid black"> </hr>

# Explore

### Univariate Exploration

In [ ]:
#visualize distribution
for col in train.columns:
    plt.hist(train[col])
    plt.title(col)
    plt.show()

In [ ]:
#specifically plot columns of interest

# Plot the distributions
plt.figure(figsize=(16, 6))

# List of columns
cols = ['Age', 'Sex', 'Race', 'MannerofDeath']

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1,4, plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    plt.suptitle('Distributions')


In [ ]:
#visualize Race data using a countplot
plt.figure(figsize=(10,6))
sns.countplot(x='Race', data=train)
plt.show()

_________________________________

### Bivariate Exploration

In [ ]:
#plot age and race
sns.swarmplot(x= train.Race, y = train.Age)
plt.title('Average Age by Race')
None

In [ ]:
#visualize correlation with Age using Heatmap
plt.figure(figsize=(6, 10))
heatmap = sns.heatmap(train.corr()[['Age']].sort_values(by='Age', ascending=False), vmin=-1, vmax=1, annot=True, cmap='mako_r')
heatmap.set_title('Features Correlating with Age', fontdict={'fontsize':18}, pad=16);

In [ ]:
#visualize Age and Herion deaths data using a countplot
plt.figure(figsize=(12,6))
sns.countplot(x='Age', data=train, hue='Heroin')
plt.title('Age of Overdose Deaths having used Heroin')
plt.show()

In [ ]:
#visualize Age and Fentanyl deaths data using a countplot
plt.figure(figsize=(12,6))
sns.countplot(x='Age', data=train, hue='Fentanyl')
plt.title('Age of Overdose Deaths having used Fentanyl')
plt.show()

In [ ]:
#visualize Age and Oxycodone deaths data using a countplot
plt.figure(figsize=(12,6))
sns.countplot(x='Age', data=train, hue='Oxycodone')
plt.title('Age of Overdose Deaths having used Oxycodone')
plt.show()

______________________

### Multivariate Exploration

In [ ]:
#visualize heroin deaths by age and race
sns.lmplot(x='Heroin', y='Age', hue='Race', data=train)
plt.title('Age of Heroin Deaths by Race')
plt.ylabel('Age')
plt.xlabel('Heroin Death (yes or no)')
plt.xlim(-0.25, 1.25)
None

In [ ]:
#visualize heroin deaths by age and race
sns.lmplot(x='White', y='Age', hue='MannerofDeath', data=train)
plt.title('Age of White Race Deaths by Manner')
plt.ylabel('Age')
plt.xlabel('Race White (yes or no)')
plt.xlim(-0.25, 1.25)
None

### Takeaways from Explore

- Heroin, Oxycodone, Black (race) have the strongest correlation with age
- The vast majority of drug overdose deaths appear to be among the white race in this dataset
- Heroin deaths peak around age 33
- Oxycodone deaths peak arond age 44
- Hispanic race appears to have the eldest drug overdose death (age 87)
- White race appears to have the youngest drug overdose death (age 14)

<hr style="border:1px solid black"> </hr>

## Hypothesis Testing

**Hypothesis 1: T-Test (continuous vs discrete)**
- $H_0$: There is a relationship between Age and White Race drug overdose deaths
- $H_a$: There is not a relationship between Age and With Race drug overdose deaths

In [ ]:
#set alpha
alpha = .05

In [ ]:
#sample size, must be more then 30 to meet assumption
train.Age.count(), train.White.count()

In [ ]:
#check variance
train.Age.var(), train.White.var()

#this shows not equal varient

In [ ]:
#t-test on entire train set
t, p = stats.ttest_ind(train.Age,train.White, equal_var=False)
t,p

In [ ]:
hypothesis_df = pd.DataFrame(data=[{
    'Hypothesis': '#1',
    'p-value': (round(p,4)),
    't-value': (round(t,3)),
    'r-value': 'N/A',
    'alpha': alpha,
    'Result': 'fail to reject $H_o$'
    }])

hypothesis_df

_______________________

**Hypothesis 2: T-Test (continuous vs discrete)**
- $H_0$: There is a relationship between Age and Oxycodone drug overdose deaths
- $H_a$: There is not a relationship between Age and Oxycodone drug overdose deaths

In [ ]:
# sample size, must be more then 30 to meet assumption
train.Age.count(), train.Oxycodone.count()

In [ ]:
#check variance
train.Age.var(), train.Oxycodone.var()

#this shows not equal varient

In [ ]:
#t-test on entire train set
t, p = stats.ttest_ind(train.Age,train.Oxycodone, equal_var=False)
t,p

In [ ]:
if p < alpha:
    print(f'We calculate a t of {t:3f} and a statistical certainty p of {p:4f}')
    print(f'Because p {p:3f} < α  {alpha:3f}, we fail to reject our null hypothesis')
else:
    print(f'We reject our null hypothesis')

In [ ]:
hypothesis_df = hypothesis_df.append(
    {
    'Hypothesis': '#2',
    'p-value': (round(p,4)),
    't-value': (round(t,4)),
    'r-value': 'N/A',
    'alpha': alpha,
    'Result': 'fail to reject $H_o$'},
    ignore_index= True
    )

hypothesis_df

_____________________________

**Hypothesis 3: T-Test (continuous vs discrete)**
- $H_0$: There is a relationship between Age and Heroin drug overdose deaths
- $H_a$: There is not a relationship between Age and Heroin drug overdose deaths

In [ ]:
# sample size, must be more then 30 to meet assumption
train.Age.count(), train.Heroin.count()

In [ ]:
#check variance
train.Age.var(), train.Heroin.var()

#this shows not equal varient

In [ ]:
#t-test on entire train set
t, p = stats.ttest_ind(train.Age,train.Heroin, equal_var=False)
t,p

In [ ]:
if p < alpha:
    print(f'We calculate a t of {t:3f} and a statistical certainty p of {p:4f}')
    print(f'Because p {p:3f} < α  {alpha:3f}, we fail to reject our null hypothesis')
else:
    print(f'We reject our null hypothesis')

In [ ]:
hypothesis_df = hypothesis_df.append(
    {
    'Hypothesis': '#2',
    'p-value': (round(p,4)),
    't-value': (round(t,4)),
    'r-value': 'N/A',
    'alpha': alpha,
    'Result': 'fail to reject $H_o$'},
    ignore_index= True
    )

hypothesis_df

__________________________

**Hypothesis 4: $X^2$ Test (discrete vs discrete)**
- $H_0$: There is a relationship between Heroin and Oxycodone drug overdose deaths
- $H_a$: There is not a relationship between Heroin and Oxycodone drug overdose deaths

In [ ]:
#pearsons correlation on entire train set
#number of rows
n = train.shape[0] 

#degrees of freedom- how much the data can vary
deg_f = n-2 

#confidence interval (!)
conf_in = 0.95

alpha = 1- conf_in

In [ ]:
x= train.Heroin
y= train.Oxycodone

In [ ]:
r, p = stats.pearsonr(x,y)
r, p

In [ ]:
if p < alpha:
    print(f'We calculate a t of {t:3f} and a statistical certainty p of {p:4f}')
    print(f'Because p {p:3f} < α  {alpha:3f}, we fail to reject our null hypothesis')
else:
    print(f'We reject our null hypothesis')

In [ ]:
hypothesis_df = hypothesis_df.append(
    {
    'Hypothesis': '#4',
    'p-value': (round(p,4)),
    't-value': 'N/A',
    'r-value': (round(r,4)),
    'alpha': alpha,
    'Result': 'fail to reject $H_o$'},
    ignore_index= True
    )

hypothesis_df

_______________________

**Hypothesis 5: $X^2$ Test (discrete vs discrete)**
- $H_0$: There is a relationship between Fentanyl and Oxycodone drug overdose deaths
- $H_a$: There is not a relationship between Fentanyl and Oxycodone drug overdose deaths

In [ ]:
#pearsons correlation on entire train set
#number of rows
n = train.shape[0] 

#degrees of freedom- how much the data can vary
deg_f = n-2 

#confidence interval (!)
conf_in = 0.95

alpha = 1- conf_in

In [ ]:
x= train.Fentanyl
y= train.Oxycodone

In [ ]:
r, p = stats.pearsonr(x,y)
r, p

In [ ]:
if p < alpha:
    print(f'We calculate a t of {t:3f} and a statistical certainty p of {p:4f}')
    print(f'Because p {p:3f} < α  {alpha:3f}, we fail to reject our null hypothesis')
else:
    print(f'We reject our null hypothesis')

In [ ]:
hypothesis_df = hypothesis_df.append(
    {
    'Hypothesis': '#5',
    'p-value': (round(p,4)),
    't-value': 'N/A',
    'r-value': (round(r,4)),
    'alpha': alpha,
    'Result': 'fail to reject $H_o$'},
    ignore_index= True
    )

hypothesis_df

____________________________________

### Takeaways from Hypothesis
- I failed to reject $H_0$ #1. Suggesting that there is a relation between age and white race drug overdose deaths
- I failed to reject $H_0$ #2. Suggesting that there is a relation between age and Oxycodone drug overdose deaths
- I failed to reject $H_0$ #3. Suggesting that there is a relation between age and Heroin drug overdose deaths
- I failed to rejected $H_0$ #4. Suggesting that there is a relationship between Heroin and Oxycodone drug overdose deaths
- I failed to rejected $H_0$ #5. Suggesting that there is a relationship between Fentanyl and Oxycodone drug overdose deaths

<hr style="border:2px solid black"> </hr>

# Model

In [ ]:
#take a look at X_train dataframe
X_train.head()

## Create Baseline

In [ ]:
#create a baseline
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

In [ ]:
# 1. Predict assessed_value_pred_mean

#predicted mean will be the mean of value_assessed column
age_pred_mean = y_train['Age'].mean()
#new column created
y_train['age_pred_mean'] = age_pred_mean
y_validate['age_pred_mean'] = age_pred_mean

In [ ]:
y_train.head()

In [ ]:
# 2. RMSE of value_assessed_pred_mean

#root mean squared error of the predicted mean
rmse_train = mean_squared_error(y_train.Age, y_train.age_pred_mean)**(1/2)
rmse_validate = mean_squared_error(y_validate.Age, y_validate.age_pred_mean)**(1/2)
print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 3), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 3))

In [ ]:
#create a dataframe to make data easier to visualize/understand
metric_df = pd.DataFrame(data=[{
    'model': 'mean_baseline',
    'rmse_train': (round(rmse_train,3)),
    'rmse_validate': (round(rmse_validate,3)),
    'r^2': (round(explained_variance_score(y_validate.Age, y_validate.age_pred_mean),3))}])

metric_df
#we now have out baseline model to work off of

_________________________

## OLS Model

In [ ]:
#set X_train to drivers only
#we do not want to include all columns in this because it could cause overfitting
#these two features have the highest correlation
drivers_train = X_train[['Oxycodone', 'Heroin']]

#set X_validate_scaled to drivers only
drivers_validate= X_validate[['Oxycodone', 'Heroin']]

#set X_test_scaled to drivers only
drivers_test= X_test[['Oxycodone', 'Heroin']]

In [ ]:
#create the model 
lm = LinearRegression(normalize=True)

#fit
lm.fit(drivers_train, y_train.Age)

In [ ]:
# predict train
y_train['age_pred_lm'] = lm.predict(drivers_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.Age, y_train.age_pred_lm)**(1/2)

# predict validate
y_validate['age_pred_lm'] = lm.predict(drivers_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.Age, y_validate.age_pred_lm)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
#create visual to see baseline vs LinearRegression model
metric_df = metric_df.append(
    {
    'model': 'OLS',
    'rmse_train': (round(rmse_train,3)),
    'rmse_validate': (round(rmse_validate,3)),
    'r^2': (round(explained_variance_score(y_validate.Age, y_validate.age_pred_lm),3))}, ignore_index=True)

metric_df

_____________________

## Lasso Lars Model

In [ ]:
# create the model object
lars = LassoLars(alpha= 2)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lars.fit(drivers_train, y_train.Age)

# predict train
y_train['age_pred_lars'] = lars.predict(drivers_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.Age, y_train.age_pred_lars)**(1/2)

# predict validate
y_validate['age_pred_lars'] = lars.predict(drivers_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.Age, y_validate.age_pred_lars)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
#shows baseline vs LinearRegression vs LassoLars
metric_df = metric_df.append(
    {
    'model': 'LassoLars (alpha 2)',
    'rmse_train': (round(rmse_train,3)),
    'rmse_validate': (round(rmse_validate,3)),
    'r^2': (round(explained_variance_score(y_validate.Age, y_validate.age_pred_lars),3))}, ignore_index=True)

metric_df

______________

## Tweedie Regressor Model

In [ ]:
# create the model object
glm = TweedieRegressor(power=1, alpha=1)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(drivers_train, y_train.Age)

# predict train
y_train['age_pred_glm'] = glm.predict(drivers_train)

# evaluate: rmse
rmse_train = (mean_squared_error(y_train.Age, y_train.age_pred_glm)**(1/2))

# predict validate
y_validate['age_pred_glm'] = glm.predict(drivers_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.Age, y_validate.age_pred_glm)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
#compares all four models
metric_df = metric_df.append(
    {
    'model': 'TweedieRegressor (power 1, alpha 2)',
    'rmse_train': (round(rmse_train,3)),
    'rmse_validate': (round(rmse_validate,3)),
    'r^2': (round(explained_variance_score(y_validate.Age, y_validate.age_pred_glm),3))}, ignore_index=True)

metric_df

____________________

In [ ]:
# plot to visualize actual vs predicted. 
plt.figure(figsize=(16,8))
plt.hist(y_validate.Age, color='blue', alpha=.5, label="Actual Age")
plt.hist(y_validate.age_pred_lm, color='red', alpha=.5, label="Model: OLS")
plt.hist(y_validate.age_pred_glm, color='yellow', alpha=.5, label="Model: TweedieRegressor")
plt.hist(y_validate.age_pred_lars, color='green', alpha=.5, label="Model: LassoLars")
plt.xlabel("Age")
plt.ylabel("Number of Homes")
plt.title("Distribution of Actual Age vs Predicted Age for Top Performing Models")
plt.legend()
plt.show()

____________________

### Use Test set on Best Performing Model

In [ ]:
y_test = pd.DataFrame(y_test)

# predict on test
y_test['age_pred_lm'] = lm.predict(drivers_test)

# evaluate: rmse
rmse_test = mean_squared_error(y_test.Age, y_test.age_pred_lm)**(1/2)

print("RMSE for OLS Model using LinearRegression\nOut-of-Sample Performance: ", rmse_test)

### Takeaways from Modeling:
- OLS model performs better than the baseline and other predictive models
- The drop of between train and validate was not significant

<hr style="border:2px solid black"> </hr>

## Final Takeaways and Next Steps:

- OLS Model performed the best, outperforming the baseline and other models
- Average age does not appear to change significantly
- I failed to reject all $H_0$.

<br>

- With more time, I would like to:
    - continue looking into my Time Series project available here: [Github Repo](https://github.com/natasharivers/individual_project/blob/main/Initial%20Time%20Series%20Notebook.ipynb)
        - Does average age change over time?
        - Do Heroin deaths increase over time?
        
<br>

- I would like to see more data
    - possibly from other areas and 
    - more recent years (2020 and 2021)